In [1]:
import os

In [5]:
pwd

'e:\\PortfolioDataScience\\End to End Projects\\Coccidiosis_Classfication_MLOps_DVC'

In [4]:
os.chdir("../")

In [6]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_url: str
    local_data_file: Path
    unzip_dir: Path

In [7]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories

In [ ]:
class ConfigurationManager:
    def __init__(self, config_filepath, params_filepath):
        self.config = read_yaml(CONFIG_FILE_PATH)
        self.params = read_yaml(PARAMS_FILE_PATH)

        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir = config.root_dir
            source_url = config.source_url
            local_data_file = config.local_data_file
            unzip_dir = config.unzip_dir
        )

        return data_ingestion_config